In [ ]:
# -------------------------------
# LSTM for Time Series Prediction
# -------------------------------

# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# -------------------------------
# 1️⃣ Load the dataset
# -------------------------------
# Assume 'dataset.csv' is uploaded and contains a single numeric column named 'value'
df = pd.read_csv('dataset.csv')

# Convert the column to float values and reshape for scaling
series = df['value'].astype('float32').values.reshape(-1, 1)

# -------------------------------
# 2️⃣ Normalize the data
# -------------------------------
# LSTMs perform better when input values are in the range [0, 1]
scaler = MinMaxScaler()
series_s = scaler.fit_transform(series)

# -------------------------------
# 3️⃣ Create input-output sequences
# -------------------------------
# Each input sequence (X) will have 'seq_len' time steps predicting the next value (y)
seq_len = 10  # You can change the number of time steps
X, y = [], []
for i in range(len(series_s) - seq_len):
    X.append(series_s[i:i + seq_len])
    y.append(series_s[i + seq_len])

# Convert lists to numpy arrays
X = np.array(X)   # Shape: (samples, seq_len, 1)
y = np.array(y)   # Shape: (samples, 1)

# -------------------------------
# 4️⃣ Split into training and testing sets
# -------------------------------
train_size = int(0.8 * len(X))  # 80% for training
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# -------------------------------
# 5️⃣ Build the LSTM model
# -------------------------------
model = Sequential([
    LSTM(32, input_shape=(seq_len, 1)),  # 32 memory units, input shape = (timesteps, features)
    Dense(1)                             # Output layer with one neuron (predicting next value)
])

# Compile with Mean Squared Error loss and Adam optimizer
model.compile(optimizer='adam', loss='mse')

# -------------------------------
# 6️⃣ Train the model
# -------------------------------
# epochs = number of passes through training data
# batch_size = number of samples per gradient update
model.fit(X_train, y_train, epochs=10, batch_size=16, validation_data=(X_test, y_test))

# -------------------------------
# 7️⃣ Make predictions and compare
# -------------------------------
pred_s = model.predict(X_test)

# Inverse transform to original scale
pred = scaler.inverse_transform(pred_s)
y_true = scaler.inverse_transform(y_test)

# -------------------------------
# 8️⃣ Display sample results
# -------------------------------
print("Predicted vs Actual values:\n")
for i in range(min(10, len(pred))):
    print(f"Predicted: {pred[i,0]:.4f} | Actual: {y_true[i,0]:.4f}")


Epoch 1/10


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 57ms/step - loss: 0.3754 - val_loss: 0.9337
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.2396 - val_loss: 0.6574
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.1764 - val_loss: 0.4269
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0907 - val_loss: 0.2417
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0529 - val_loss: 0.1010
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0201 - val_loss: 0.0215
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0095 - val_loss: 0.0025
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0130 - val_loss: 0.0032
Epoch 9/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.0139 - val_loss: 0.0027
Epoch 10/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0090 - val_loss: 0.0085
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
Predicted vs Actual values:

Predicted: 127.3720 | Actual: 124.2626
Predicted: 128.2852 | Actual: 140.6077
Predicted: 127.7287 | Actual: 134.